Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[ignite,pyyaml]"

## Setup imports

In [ ]:
from monai.config import print_config

print_config()

# MONAI Bundles

Bundles are essentially _self-descriptive networks_. They combine a network definition with the metadata about what they are meant to do, what they are used for, the nature of their inputs and outputs, and scripts (possibly with associated data) to train and infer using them. 

The key objective with bundles is to provide a structured format for using and distributing your network along with all the added information needed to understand the network in context. This makes it easier for you and others to use the network, adapt it to different applications, reproduce your experiments and results, and simply document your work.

The bundle documentation and specification can be found here: https://docs.monai.io/en/stable/bundle_intro.html

## Bundle Structure

A bundle consists of a named directory containing specific subdirectories for different parts. From the specification we have a basic outline of directories in this form (* means optional file):

```
ModelName
┣━ configs
┃  ┗━ metadata.json
┣━ models
┃  ┣━ model.pt
┃  ┣━ *model.ts
┃  ┗━ *model.onnx
┣━ docs
┃  ┣━ *README.md
┃  ┗━ *license.txt
┗━ *scripts
```

Here the `metadata.json` file will contain the name of the bundle, plain language description of what it does and intended purpose, a description of what the input and output values are for the network's forward pass, copyright information, and otherwise anything else you want to add. Further configuration files go into `configs` which will be JSON or YAML documents representing scripts in the form of Python object instantiations.

The `models` directory contains the stored weights for your network which can be in multiple forms. The weight dictionary `model.pt` must be present but the Torchscript `model.ts` and ONNX `model.onnx` files representing the same network are optional. 

The `docs` directory will contain the readme file and any other documentation you want to include. Notebooks and images are good things to include for demonstrating use of the bundle.

A further `scripts` directory can be included which would contain Python definitions of any sort to be used in the JSON/YAML script files. This directory should be a valid Python module if present, ie. contains a `__init__.py` file.

## Instantiating a new bundle

This notebook will introduce the concepts of the bundle and how to define your own. MONAI provides a number of bundle-related programs through the `monai.bundle` module using the Fire library. We can use `init_bundle` to start creating a bundle from scratch:

In [1]:
%%bash

python -m monai.bundle init_bundle TestBundle
# you may need to install tree with "sudo apt install tree"
which tree && tree TestBundle || true

TestBundle
├── configs
│   ├── inference.json
│   └── metadata.json
├── docs
│   └── README.md
├── LICENSE
└── models

3 directories, 4 files


Our new blandly-named bundle, `TestBundle`, doesn't have much in it currently. It has the directory structure so we can start putting definitions in the right places. The first thing we should do is fill in relevant information to the `metadata.json` file so that anyone who has our bundle knows what it is. The default is a template of common fields:

In [2]:
!cat TestBundle/configs/metadata.json

{
    "version": "0.0.1",
    "changelog": {
        "0.0.1": "Initial version"
    },
    "monai_version": "1.2.0",
    "pytorch_version": "2.0.0",
    "numpy_version": "1.23.5",
    "optional_packages_version": {},
    "task": "Describe what the network predicts",
    "description": "A longer description of what the network does, use context, inputs, outputs, etc.",
    "authors": "Your Name Here",
    "copyright": "Copyright (c) Your Name Here",
    "network_data_format": {
        "inputs": {},
        "outputs": {}
    }
}

We'll replace this with some more information that reflects our bundle being a demo:

In [2]:
%%writefile TestBundle/configs/metadata.json

{
    "version": "0.0.1",
    "changelog": {
        "0.0.1": "Initial version"
    },
    "monai_version": "1.2.0",
    "pytorch_version": "2.0.0",
    "numpy_version": "1.23.5",
    "optional_packages_version": {},
    "name": "TestBundle",
    "task": "Demonstration Bundle Network",
    "description": "This is a demonstration bundle meant to showcase features of the MONAI bundle system only and does nothing useful",
    "authors": "Your Name Here",
    "copyright": "Copyright (c) Your Name Here",
    "network_data_format": {
        "inputs": {},
        "outputs": {}
    },
    "intended_use": "This is suitable for demonstration only"
}

Overwriting TestBundle/configs/metadata.json


## Configuration Files

Configuration files define how to instantiate a number of Python objects and run simple routines. These files, whether JSON or YAML, are Python dictionaries containing expression lists or the arguments to be passed to a named constructor.

The provided `inference.json` file is a demo of applying a network to a series of JPEG images. This illustrates some of the concepts around typical bundles, specifically how to declare MONAI objects to put a workflow together, but we're going to ignore that for now and create some YAML configuration files instead which do some very basic things. 

Whether you're working with JSON or YAML the config files are doing the same thing which is define a series of object instantiations with the expectation that this constitutes a workflow. Typically for training or inference with a network this would be defining data sources, loaders, transform sequences, and finally a subclass of the [Ignite Engine](https://docs.monai.io/en/stable/engines.html#workflow). A class like `SupervisedTrainer` is the driving program for training a network, so creating an instance of this along with its associated arguments then calling its `run()` method constitutes a workflow or "program". 

You don't have to use any specific objects types though so you're totally free to design your workflows to be whatever you like, but typically as demonstrated in the MONAI Model Zoo they'll be Ignite-based workflows doing training or inference. We'll start with a very simple workflow which actually just imports Pytorch and MONAI then prints diagnostic information:

In [32]:
%%writefile TestBundle/configs/test_config.yaml

imports: 
- $import torch
- $import monai

device: $torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

shape: [4, 4]

test_tensor: '$torch.rand(*@shape).to(@device)'

test_config:
- '$monai.config.print_config()'
- '$print("Test tensor:", @test_tensor)'

Writing TestBundle/configs/test_config.yaml


This file demonstrates a number of key concepts:

* `imports` is a sequence of strings starting with `$` which indicate the string should be interpreted as a Python expression. These will be interpreted at the start of the execution so that modules can be imported into the running namespace. `imports` should be a sequence of such expressions.
* `device` is an object definition created by evaluating the given expression, in this case creating a Pytorch device object.
* `shape` is a list of literal values in YAML format we'll use elsewhere.
* `test_tensor` is another object created by evaluating an expression, this one uses references to `shape` and `device` with the `@` syntax.
* `test_config` is a list of expressions which are evaluated in order to act as the "main" or entry point for the program, in this case printing config information and then our created tensor.

As mentioned `$` and `@` are sigils with special meaning. A string starting with `$` is treated as a Python expression and is evaluated as such when needed, these need to be enclosed in quotes only when JSON/YAML need that to parse correctly. A variable starting with `@` is treated as reference to something we've defined in the script, eg `@shape`, and will only work for such definitions. Accessing a member of a definition before being interpreted can be done with `#`, so something like `@foo#bar` will access the `bar` member of a definition `foo`. More information on the usage of these can be found at https://docs.monai.io/en/latest/config_syntax.html.

We can run this "program" on the command line now using the bundle submodule and a few arguments to specify the metadata file and configuration file:

In [33]:
%%bash

# convenient to define the bundle's root in a variable
BUNDLE="./TestBundle"

# loads the test_config.yaml file and runs the test_config program it defines
python -m monai.bundle run test_config \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/test_config.yaml"

2023-07-14 15:34:52,646 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-07-14 15:34:52,647 - INFO - > run_id: 'test_config'
2023-07-14 15:34:52,647 - INFO - > meta_file: './TestBundle/configs/metadata.json'
2023-07-14 15:34:52,647 - INFO - > config_file: './TestBundle/configs/test_config.yaml'
2023-07-14 15:34:52,647 - INFO - ---




/home/localek10/workspace/monai/MONAI_mine/monai/bundle/workflows.py:213: UserWarning: Default logging file in 'configs/logging.conf' does not exist, skipping logging.
  warnings.warn("Default logging file in 'configs/logging.conf' does not exist, skipping logging.")


MONAI version: 1.2.0
Numpy version: 1.23.5
Pytorch version: 2.0.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /home/localek10/workspace/monai/MONAI_mine/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.12
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.0.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.4.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.0
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.5.3
einops version: 0.6.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/la

Here the `run` routine is invoked and the name of the "main" sequence of expressions is given (`test_config`). MONAI will then load and interpret the config then evaluate the expressions of `test_config` in order. Definitions in the configuratoin which aren't needed to do this are ignored, so you can provide multiple expression lists that run different parts of your script without having to create everything. 

## Object Instantiation

Creating objects is a key concept in config files which would be cumbersome if done only through expressions as has been demonstrated here. Instead, an object can be defined by a dictionary of values naming first the type with `_target_` and then providing the constructor arguments as named values. The following is a simple example creat a `Dataset` class with a very simple set of values:

In [30]:
%%writefile TestBundle/configs/test_object.yaml

datadicts: '$[{i: (i * i)} for i in range(10)]'  # create a fake dataset as a list of dicts

test_dataset:  # creates an instance of an object because _target_ is present
  _target_: Dataset  # name of type to create is monai.data.Dataset (loaded implicitly from MONAI)
  data: '@datadicts'  # argument data provided by a definition
  transform: '$None'  # argument transform provided by a Python expression

test:
- '$print("Dataset", @test_dataset)'
- '$print("Size", len(@test_dataset))'
- '$print("Transform member", @test_dataset.transform)'
- '$print("Values", list(@test_dataset))'

Overwriting TestBundle/configs/test_object.yaml


In [31]:
%%bash

BUNDLE="./TestBundle"

# prints normal values
python -W ignore -m monai.bundle run test \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/test_object.yaml"

2023-07-14 15:28:36,063 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-07-14 15:28:36,063 - INFO - > run_id: 'test'
2023-07-14 15:28:36,063 - INFO - > meta_file: './TestBundle/configs/metadata.json'
2023-07-14 15:28:36,063 - INFO - > config_file: './TestBundle/configs/test_object.yaml'
2023-07-14 15:28:36,063 - INFO - ---


Dataset <monai.data.dataset.Dataset object at 0x7fa6d117f100>
Size 10
Transform member None
Values [{0: 0}, {1: 1}, {2: 4}, {3: 9}, {4: 16}, {5: 25}, {6: 36}, {7: 49}, {8: 64}, {9: 81}]


The `test_dataset` definition is roughly equivalent to the expression `Dataset(data=datadicts, transform=None)`. Like regular Python we don't need to provide values for arguments having defaults, but we can only give argument values by name and not by position. 

## Command Line Definitions

Command line arguments can be provided to add or modify definitions in the script you're running. Using `--` before the name of the variable allows you to set their value with the next argument, but this must be a valid Python expression. You can also set individual members of definitions with `#` but be sure to put quotes around the argument in Bash. 

We can demo this with an even simpler script:

In [20]:
%%writefile TestBundle/configs/test_cmdline.yaml

shape: [8, 8]
area: '$@shape[0]*@shape[1]'

test:
- '$print("Height", @shape[0])'
- '$print("Width", @shape[1])'
- '$print("Area", @area)'

Writing TestBundle/configs/test_cmdline.yaml


In [21]:
%%bash

BUNDLE="./TestBundle"

# prints normal values
python -W ignore -m monai.bundle run test \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/test_cmdline.yaml"

# half the height
python -W ignore -m monai.bundle run test \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/test_cmdline.yaml" \
    '--shape#0' 4

# area definition replaces existing expression with a lie
python -W ignore -m monai.bundle run test \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/test_cmdline.yaml" \
    --area 32

2023-07-14 15:22:37,435 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-07-14 15:22:37,435 - INFO - > run_id: 'test'
2023-07-14 15:22:37,436 - INFO - > meta_file: './TestBundle/configs/metadata.json'
2023-07-14 15:22:37,436 - INFO - > config_file: './TestBundle/configs/test_cmdline.yaml'
2023-07-14 15:22:37,436 - INFO - ---


Height 8
Width 8
Area 64
2023-07-14 15:22:40,876 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-07-14 15:22:40,876 - INFO - > run_id: 'test'
2023-07-14 15:22:40,876 - INFO - > meta_file: './TestBundle/configs/metadata.json'
2023-07-14 15:22:40,876 - INFO - > config_file: './TestBundle/configs/test_cmdline.yaml'
2023-07-14 15:22:40,876 - INFO - > shape#0: 4
2023-07-14 15:22:40,876 - INFO - ---


Height 4
Width 8
Area 32
2023-07-14 15:22:44,279 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-07-14 15:22:44,279 - INFO - > run_id: 'test'
2023-07-14 15:22:44,279 - INFO - > meta_file: './TestBundle/configs/metadata.json'


## Multiple Files

Multiple config files can be specified which will create a final script composed of definitions in the first file added to or updated with those in subsequent files. Remember that the files are essentially creating Python dictionaries of definitions that are interpreted later, so later files are just updating that dictionary when loaded. Definitions in one file can be referenced in others:

In [17]:
%%writefile TestBundle/configs/multifile1.yaml

width: 8
height: 8

Writing TestBundle/configs/multifile1.yaml


In [18]:
%%writefile TestBundle/configs/multifile2.yaml

area: '$@width*@height'

test:
- '$print("Area", @area)'

Writing TestBundle/configs/multifile2.yaml


In [19]:
%%bash

BUNDLE="./TestBundle"

# area definition replaces existing expression with a lie
python -W ignore -m monai.bundle run test \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "['$BUNDLE/configs/multifile1.yaml','$BUNDLE/configs/multifile2.yaml']"

2023-07-14 15:09:59,663 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-07-14 15:09:59,663 - INFO - > run_id: 'test'
2023-07-14 15:09:59,663 - INFO - > meta_file: './TestBundle/configs/metadata.json'
2023-07-14 15:09:59,663 - INFO - > config_file: ['./TestBundle/configs/multifile1.yaml', './TestBundle/configs/multifile2.yaml']
2023-07-14 15:09:59,663 - INFO - ---


Area 64


The value for `config_file` in this example is a Python list containing 2 strings. It takes a bit of care to get the Bash syntax right so that this expression isn't mangled (eg. avoid spaces to prevent tokenisation and use "" quotes so that other quotes aren't interpreted), but is otherwise a simple mechanism.

This mechanism, and the ability to add/modify definitions on the command line, is important for a number of reasons:

* It lets you write a "common" configuration file containing definitions to be used with other config files and so reduce duplication.
* It lets different expressions or setups to be defined with different combinations of files, again avoiding having to duplicate then modify scripts for different experiments.
* Adding/changing definitions also allows quick minor changes or batching of different operations on the command line or in shell scripts, eg. doing a parameter sweep by looping through possible values and passing them as arguments. 

## Summary and Next

We have here described the basics of bundles:

* Directory structure
* Metadata file
* Configuration files
* Command line usage

In the next tutorial we will actually create a bundle for a real network that does something and demonstrate features for working with networks.